In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [4]:
%%px
# Import the libraries

from numpy import *
from time import time
from mpi4py import MPI
from mpiFFT4py.slab import R2C

In [5]:
%%px

# Set global size of the computational box
M = 6
N = array([2**M, 2**M, 2**M], dtype=int)
L = array([2*pi, 2*pi, 2*pi], dtype=float)

In [6]:
%%px 

FFT = R2C(N, L, MPI.COMM_WORLD, "double", communication='Alltoallw')

In [7]:
%%px

U = random.random(FFT.real_shape()).astype(FFT.float) # real_shape = (N[0]//comm.Get_size(), N[1], N[2])
U_copy = zeros_like(U)
U_hat = zeros(FFT.complex_shape(), dtype=FFT.complex) # complex_shape = (N[0], N[1]//comm.Get_size(), N[2]//2+1)

In [12]:
%%px

U_hat = FFT.fftn(U, U_hat)

In [13]:
%%px

U_copy = FFT.ifftn(U_hat, U_copy)
MPI.COMM_WORLD.barrier()

In [15]:
%%px 

t0 = time()
U_hat = FFT.fftn(U, U_hat)
U_copy = FFT.ifftn(U_hat, U_copy)
print("mpiFFT4py ", time()-t0)

[stdout:0] mpiFFT4py  0.16920161247253418
[stdout:1] mpiFFT4py  0.17447113990783691
[stdout:2] mpiFFT4py  0.16347527503967285
[stdout:3] mpiFFT4py  0.16394305229187012
[stdout:4] mpiFFT4py  0.1016395092010498
[stdout:5] mpiFFT4py  0.09713125228881836
[stdout:6] mpiFFT4py  0.10159826278686523
[stdout:7] mpiFFT4py  0.08875775337219238


In [16]:
%%px

tol = 1e-6 if FFT.float == float32 else 1e-10

In [19]:
%%px

assert allclose(U, U_copy, tol, tol)